## Libraries

In [2]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings
import psycopg2

# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = '{:.3f}'.format

df_sc = statcast(start_dt='2024-01-01', end_dt='2024-09-30')



This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 200/200 [02:23<00:00,  1.39it/s]
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


## Batter Info

In [ ]:
# Step 1: Your existing unique count
df_unique = df_sc[['game_year', 'batter', 'stand', 'game_pk', 'at_bat_number']].drop_duplicates()
result = df_unique.groupby(['game_year', 'batter', 'stand']).size().reset_index(name='num_at_bats')

# Step 2: Pivot to check counts for both stands
pivot = result.pivot(index=['game_year', 'batter'], columns='stand', values='num_at_bats').fillna(0)

# Step 3: Apply logic for labeling
def label_row(row):
    if row.get('L', 0) > 5 and row.get('R', 0) > 5:
        return 'B'
    elif row.get('L', 0) > row.get('R', 0):
        return 'L'
    else:
        return 'R'

pivot['bats'] = pivot.apply(label_row, axis=1)

# Step 4: Merge back with your original result
final_result = result.merge(pivot[['bats']], on='batter')

batter_stand_info = final_result[['game_year', 'batter', 'bats']].drop_duplicates()

batter_stand_info.to_csv('batter_info.csv', index=False)
batter_stand_info

# psql -h piedmont-db.cp6uayweg6ql.us-east-2.rds.amazonaws.com -U postgres -d postgres -p 5432
# \COPY batter_info(game_year, batter, bats) FROM '/Users/loganmottley/Desktop/Projects/go-baseball/data/batter_info.csv' DELIMITER ',' CSV HEADER;

,game_year,batter,bats
0,2024,444482,L
1,2024,453568,L
2,2024,455117,R
3,2024,456781,R
4,2024,457705,R
...,...,...,...
1540,2024,814816,R
1541,2024,815151,R
1542,2024,815694,R
1543,2024,815842,R


## Swing Decision

In [56]:
swing_events = [
    'swinging_strike', 'swinging_strike_blocked', 'foul', 
    'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip', 'hit_into_play'
]

df_sc['is_swing'] = df_sc['description'].isin(swing_events).astype(int)


# Group by batter, stand, p_throws, zone, pitch_type
group_cols = ['game_year', 'batter', 'stand', 'p_throws', 'zone', 'pitch_type']

# Calculate total pitches and total swings
swing_summary = df_sc.groupby(group_cols).agg(
    total_pitches=('is_swing', 'count'),
    total_swings=('is_swing', 'sum')
).reset_index()

# Calculate swing percentage
swing_summary['swing_percentage'] = swing_summary['total_swings'] / swing_summary['total_pitches']



swing_summary.to_csv('batter_swing_percentage.csv', index=False)


## Contact

In [6]:
# Define contact categories
def classify_contact(desc):
    if desc in ['swinging_strike', 'swinging_strike_blocked', 'missed_bunt']:
        return 'swinging_strike'
    elif desc in ['foul', 'foul_tip', 'foul_bunt', 'bunt_foul_tip']:
        return 'foul'
    elif desc == 'hit_into_play':
        return 'ball_in_play'
    else:
        return None  # ignore non-swings like 'ball', 'called_strike'

df_sc['is_contact'] = df_sc['description'].apply(classify_contact)

# Only keep rows where batter swung
df_swing = df_sc[df_sc['is_contact'].notnull()]

# Group by batter, stand, p_throws, zone, pitch_type
group_cols = ['game_year', 'batter', 'stand', 'p_throws', 'zone', 'pitch_type']

# Calculate counts for each contact type
contact_counts = df_swing.groupby(group_cols + ['is_contact']).agg(
    count=('description', 'count')
).reset_index()

# Pivot to wide format
pivot = contact_counts.pivot_table(index=group_cols, columns='is_contact', values='count', fill_value=0).reset_index()

# Calculate total swings and percentages
pivot['total_swings'] = pivot[['swinging_strike', 'foul', 'ball_in_play']].sum(axis=1)
pivot['pct_swinging_strike'] = pivot['swinging_strike'] / pivot['total_swings']
pivot['pct_foul'] = pivot['foul'] / pivot['total_swings']
pivot['pct_ball_in_play'] = pivot['ball_in_play'] / pivot['total_swings']

pivot['swinging_strike'] = pivot['swinging_strike'].astype(int)
pivot['foul'] = pivot['foul'].astype(int)
pivot['ball_in_play'] = pivot['ball_in_play'].astype(int)
pivot['total_swings'] = pivot['total_swings'].astype(int)


# Optional: export
pivot.to_csv('batter_contact_percentages.csv', index=False)

# \COPY batter_contact_percentage(game_year, batter, stand, p_throws, zone, pitch_type, swinging_strike, foul, ball_in_play, total_swings, pct_swinging_strike, pct_foul, pct_ball_in_play) FROM '/Users/loganmottley/Desktop/Projects/go-baseball/data/batter_contact_percentages.csv' DELIMITER ',' CSV HEADER;


## Precomputed Probabilities 

In [44]:

df_sc_filtered = df_sc[(df_sc['type'] == 'X')]



In [46]:
df_sc_filtered = df_sc[(df_sc['type'] == 'X')]

# Create velocity buckets
bins = [0, 70, 75, 80, 85, 90, 95, 100, 105]
labels = ['<70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100', '100+']
df_sc_filtered['velocity_bucket'] = pd.cut(df_sc_filtered['release_speed'], bins=bins, labels=labels, right=False)

# Simplify outcomes
df_sc_filtered['outcome'] = np.where(df_sc_filtered['events'].isin([
    'field_out', 'force_out', 'grounded_into_double_play', 'sac_fly', 'field_error',
    'sac_bunt', 'fielders_choice', 'double_play', 'fielders_choice_out', 
    'sac_fly_double_play', 'triple_play'
]), 'out', df_sc_filtered['events'])

df_sc_filtered['count_state'] = np.where(
    df_sc_filtered['balls'] == df_sc_filtered['strikes'], 'even',
    np.where(df_sc_filtered['balls'] < df_sc_filtered['strikes'], 'ahead', 'behind')
)


# Group 1
batter_pitch_type = df_sc_filtered.groupby(['game_year', 'batter', 'stand', 'pitch_type', 'outcome']).size().reset_index(name='count')
batter_pitch_type['prob'] = batter_pitch_type['count'] / batter_pitch_type.groupby(['game_year', 'batter', 'stand', 'pitch_type'])['count'].transform('sum')

# Group 2
batter_zone = df_sc_filtered.groupby(['game_year', 'batter', 'stand', 'zone', 'outcome']).size().reset_index(name='count')
batter_zone['prob'] = batter_zone['count'] / batter_zone.groupby(['game_year', 'batter', 'stand', 'zone'])['count'].transform('sum')

# Group 3
batter_velo = df_sc_filtered.groupby(['game_year', 'batter', 'stand', 'velocity_bucket', 'outcome']).size().reset_index(name='count')
batter_velo['prob'] = batter_velo['count'] / batter_velo.groupby(['game_year', 'batter', 'stand', 'velocity_bucket'])['count'].transform('sum')

# Group 4
batter_pthrows = df_sc_filtered.groupby(['game_year', 'batter', 'stand', 'p_throws', 'outcome']).size().reset_index(name='count')
batter_pthrows['prob'] = batter_pthrows['count'] / batter_pthrows.groupby(['game_year', 'batter', 'stand', 'p_throws'])['count'].transform('sum')


/var/folders/fl/cwc6990j59q5wy2jkl371h040000gn/T/ipykernel_1299/1031727184.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sc_filtered['velocity_bucket'] = pd.cut(df_sc_filtered['release_speed'], bins=bins, labels=labels, right=False)
/var/folders/fl/cwc6990j59q5wy2jkl371h040000gn/T/ipykernel_1299/1031727184.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sc_filtered['outcome'] = np.where(df_sc_filtered['events'].isin([
/var/folders/fl/cwc6990j59q5wy2jkl371h040000gn/T/ipykernel_1299/1031727184

In [51]:
def lookup_fixed_levels(query):
    game_year = query['game_year']
    batter = query['batter']
    stand = query['stand']
    pitch_type = query['pitch_type']
    zone = query['zone']
    p_throws = query['p_throws']
    velocity_bucket = query['velocity_bucket']

    results = {}

    # Add n to each slice
    level_pitch_type = batter_pitch_type[
        (batter_pitch_type['game_year'] == game_year) &
        (batter_pitch_type['batter'] == batter) &
        (batter_pitch_type['stand'] == stand) &
        (batter_pitch_type['pitch_type'] == pitch_type)
    ]
    results['batter_stand_pitch_type'] = {
        "n": int(level_pitch_type['count'].sum()) if not level_pitch_type.empty else 0,
        "probs": level_pitch_type[['outcome', 'prob']].set_index('outcome')['prob'].to_dict() if not level_pitch_type.empty else {}
    }

    level_zone = batter_zone[
        (batter_zone['game_year'] == game_year) &
        (batter_zone['batter'] == batter) &
        (batter_zone['stand'] == stand) &
        (batter_zone['zone'] == zone)
    ]
    results['batter_stand_zone'] = {
        "n": int(level_zone['count'].sum()) if not level_zone.empty else 0,
        "probs": level_zone[['outcome', 'prob']].set_index('outcome')['prob'].to_dict() if not level_zone.empty else {}
    }

    level_velo = batter_velo[
        (batter_velo['game_year'] == game_year) &
        (batter_velo['batter'] == batter) &
        (batter_velo['stand'] == stand) &
        (batter_velo['velocity_bucket'] == velocity_bucket)
    ]
    results['batter_stand_velocity_bucket'] = {
        "n": int(level_velo['count'].sum()) if not level_velo.empty else 0,
        "probs": level_velo[['outcome', 'prob']].set_index('outcome')['prob'].to_dict() if not level_velo.empty else {}
    }

    level_pthrows = batter_pthrows[
        (batter_pthrows['game_year'] == game_year) &
        (batter_pthrows['batter'] == batter) &
        (batter_pthrows['stand'] == stand) &
        (batter_pthrows['p_throws'] == p_throws)
    ]
    results['batter_stand_p_throws'] = {
        "n": int(level_pthrows['count'].sum()) if not level_pthrows.empty else 0,
        "probs": level_pthrows[['outcome', 'prob']].set_index('outcome')['prob'].to_dict() if not level_pthrows.empty else {}
    }

    return results


In [52]:
from tqdm import tqdm

# Get all unique batter + game_year + stand combos
combo_cols = ['game_year', 'batter', 'stand']
unique_combos = df_sc_filtered[combo_cols].drop_duplicates()

batch_results = []

for _, row in tqdm(unique_combos.iterrows(), total=unique_combos.shape[0], desc="Processing batters"):
    query = {
        'game_year': row['game_year'],
        'batter': row['batter'],
        'stand': row['stand'],
        'pitch_type': None,
        'zone': None,
        'p_throws': None,
        'velocity_bucket': None
    }

    # Expand by pitch_type
    for pitch_type in tqdm(df_sc_filtered['pitch_type'].unique(), desc=f"  Expanding pitch_types for batter {row['batter']}", leave=False):
        query['pitch_type'] = pitch_type
        output = lookup_fixed_levels(query)
        batch_results.append({
            **query,
            **output['batter_stand_pitch_type']['probs'],
            'n': output['batter_stand_pitch_type']['n'],
            'level': 'batter_stand_pitch_type'
        })

    # Expand by zone
    for zone in tqdm(df_sc_filtered['zone'].unique(), desc=f"  Expanding zones for batter {row['batter']}", leave=False):
        query['zone'] = zone
        output = lookup_fixed_levels(query)
        batch_results.append({
            **query,
            **output['batter_stand_zone']['probs'],
            'n': output['batter_stand_zone']['n'],
            'level': 'batter_stand_zone'
        })

    # Expand by velocity bucket
    for velo in tqdm(df_sc_filtered['velocity_bucket'].unique(), desc=f"  Expanding velo for batter {row['batter']}", leave=False):
        query['velocity_bucket'] = velo
        output = lookup_fixed_levels(query)
        batch_results.append({
            **query,
            **output['batter_stand_velocity_bucket']['probs'],
            'n': output['batter_stand_velocity_bucket']['n'],
            'level': 'batter_stand_velocity_bucket'
        })

    # Expand by p_throws
    for p_throw in tqdm(df_sc_filtered['p_throws'].unique(), desc=f"  Expanding p_throws for batter {row['batter']}", leave=False):
        query['p_throws'] = p_throw
        output = lookup_fixed_levels(query)
        batch_results.append({
            **query,
            **output['batter_stand_p_throws']['probs'],
            'n': output['batter_stand_p_throws']['n'],
            'level': 'batter_stand_p_throws'
        })

# Convert to dataframe
full_table = pd.DataFrame(batch_results)


Processing batters: 100%|██████████| 1376/1376 [05:47<00:00,  3.96it/s]


In [56]:
# Define the desired column order
column_order = [
    'game_year', 
    'batter', 
    'stand', 
    'pitch_type', 
    'zone', 
    'p_throws', 
    'velocity_bucket', 
    'double', 
    'home_run', 
    'out', 
    'single', 
    'triple', 
    'n', 
    'level'
]

# Ensure missing columns (if any) are added with default values
for col in column_order:
    if col not in full_table.columns:
        full_table[col] = None

# Reorder
full_table = full_table[column_order]

# Optional: Export to CSV
full_table.to_csv('/Users/loganmottley/Desktop/Projects/go-baseball/data/batter_hit_type.csv', index=False)


In [55]:
full_table


,game_year,batter,stand,pitch_type,zone,p_throws,velocity_bucket,double,home_run,out,single,n,level,triple
0,2024,518595,R,FF,None,None,None,0.100000,0.071429,0.657143,0.171429,70,batter_stand_pitch_type,NaN
1,2024,518595,R,SL,None,None,None,0.100000,0.133333,0.700000,0.066667,30,batter_stand_pitch_type,NaN
2,2024,518595,R,CU,None,None,None,NaN,0.166667,0.666667,0.166667,6,batter_stand_pitch_type,NaN
3,2024,518595,R,SI,None,None,None,0.015873,0.015873,0.682540,0.285714,63,batter_stand_pitch_type,NaN
4,2024,518595,R,FC,None,None,None,NaN,0.071429,0.857143,0.071429,14,batter_stand_pitch_type,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57787,2024,689531,L,FO,<NA>,None,100+,NaN,NaN,NaN,NaN,0,batter_stand_velocity_bucket,NaN
57788,2024,689531,L,FO,<NA>,None,<70,NaN,NaN,NaN,NaN,0,batter_stand_velocity_bucket,NaN
57789,2024,689531,L,FO,<NA>,None,NaN,NaN,NaN,NaN,NaN,0,batter_stand_velocity_bucket,NaN
57790,2024,689531,L,FO,<NA>,R,NaN,NaN,NaN,NaN,NaN,0,batter_stand_p_throws,NaN


In [ ]:

# \COPY batter_hit_type(game_year, batter, stand, pitch_type, zone, p_throws, velocity_bucket, double, home_run, out, single, triple, n, level) FROM '/Users/loganmottley/Desktop/Projects/go-baseball/data/batter_hit_type.csv' DELIMITER ',' CSV HEADER;



,game_year,batter,stand,pitch_type,zone,p_throws,velocity_bucket,double,home_run,out,single,n,level,triple
0,2024,518595,R,FF,None,None,None,0.100000,0.071429,0.657143,0.171429,70,batter_stand_pitch_type,NaN
1,2024,518595,R,SL,None,None,None,0.100000,0.133333,0.700000,0.066667,30,batter_stand_pitch_type,NaN
2,2024,518595,R,CU,None,None,None,NaN,0.166667,0.666667,0.166667,6,batter_stand_pitch_type,NaN
3,2024,518595,R,SI,None,None,None,0.015873,0.015873,0.682540,0.285714,63,batter_stand_pitch_type,NaN
4,2024,518595,R,FC,None,None,None,NaN,0.071429,0.857143,0.071429,14,batter_stand_pitch_type,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57787,2024,689531,L,FO,<NA>,None,100+,NaN,NaN,NaN,NaN,0,batter_stand_velocity_bucket,NaN
57788,2024,689531,L,FO,<NA>,None,<70,NaN,NaN,NaN,NaN,0,batter_stand_velocity_bucket,NaN
57789,2024,689531,L,FO,<NA>,None,NaN,NaN,NaN,NaN,NaN,0,batter_stand_velocity_bucket,NaN
57790,2024,689531,L,FO,<NA>,R,NaN,NaN,NaN,NaN,NaN,0,batter_stand_p_throws,NaN
